# Mapa de Oferta Alimentaria Saludable en Lima, Peru

In [ ]:
# Dependencias
import pandas as pd
import numpy as np
import geopandas as gpd
import urbanpy as up
import contextily as cx
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.lines import Line2D

## Bioferias

### Ferias organizadas por promotores que controlan una oferta realmente ecológica:

- Feria Ecológica de Barranco (Calle San Martín cuadra 7, Barranco / domingos), -12.138176, -77.0229817
- Ecoferia de Cieneguilla (Frente parque Simón Bolívar; III etapa Cieneguilla / domingos), -12.0943875, -76.7749452
- BioFeria de Miraflores (parque Reducto / sábados), -12.1270848, -77.022242
- BioFeria de Surquillo (jirón Narciso de la Colina 580 / domingos), -12.118381, -77.025619

### Feria que escoge una oferta saludable combinando entre ecológicos certificados y convencionales - AgroFerias Campesinas:

- sede Puericultorio Pérez Araníbar (jirón Antonio Raymondi 141, Magdalena del Mar / viernes, sábados, domingos), -12.0990915, -77.06884
- sede San Isidro (avenida Aramburú cruce con calle 3, San Isidro / sábados), -12.1021172, -77.0162924
- sede Parque de la Familia, San Borja (Av. San Borja Sur con jirón Frederic Remington / domingos), -12.098735230950824, -77.01049887420028

### Bioferias de vecindarios alimentarios:

- BioFeria Casablanca, Pachacamac  (2do sabado / mensual)
- BioFeria Tungasuca, Carabayllo (sabado / semanal)

In [ ]:
bioferias = [
    {
        "nombre": "Feria Ecológica de Barranco",
        "direccion": "Calle San Martín cuadra 7, Barranco",
        "temporalidad": "domingos",
        "observaciones": "Ferias organizadas por promotores que controlan una oferta realmente ecológica",
        "lat": -12.138176,
        "lon": -77.0229817,
    },
    {
        "nombre": "Ecoferia de Cieneguilla",
        "direccion": "Frente parque Simón Bolívar; III etapa Cieneguilla",
        "temporalidad": "domingos",
        "observaciones": "Ferias organizadas por promotores que controlan una oferta realmente ecológica",
        "lat": -12.0943875,
        "lon": -76.7749452,
    },
    {
        "nombre": "BioFeria de Miraflores",
        "direccion": "Parque Reducto",
        "temporalidad": "sábados",
        "observaciones": "Ferias organizadas por promotores que controlan una oferta realmente ecológica",
        "lat": -12.1270848,
        "lon": -77.022242,
    },
    {
        "nombre": "BioFeria de Surquillo",
        "direccion": "Jirón Narciso de la Colina 580",
        "temporalidad": "domingos",
        "observaciones": "Ferias organizadas por promotores que controlan una oferta realmente ecológica",
        "lat": -12.118381,
        "lon": -77.025619,
    },
    {
        "nombre": "AgroFeria Campesina",
        "direccion": "Jirón Antonio Raymondi 141, Magdalena del Mar",
        "temporalidad": "viernes, sábados y domingos",
        "observaciones": "Feria que escoge una oferta saludable combinando entre ecológicos certificados y convencionales",
        "lat": -12.0990915,
        "lon": -77.06884,
    },
    {
        "nombre": "AgroFeria Campesina",
        "direccion": "Avenida Aramburú cruce con calle 3, San Isidro",
        "temporalidad": "sábados",
        "observaciones": "Feria que escoge una oferta saludable combinando entre ecológicos certificados y convencionales",
        "lat": -12.1021172,
        "lon": -77.0162924,
    },
    {
        "nombre": "AgroFeria Campesina",
        "direccion": "Avenida San Borja Sur con Jirón Frederic Remington",
        "temporalidad": "domingos",
        "observaciones": "Feria que escoge una oferta saludable combinando entre ecológicos certificados y convencionales",
        "lat": -12.098735230950824,
        "lon": -77.01049887420028,
    },
    {"nombre": "BioFeria Casablanca", "lat": -12.205550, "lon": -76.866392},
    {"nombre": "BioFeria Tungasuca", "lat": -11.89664, "lon": -77.04062},
    {"nombre": "BioFeria Picapiedra Pachacamac", "lat": -12.186558, "lon": -76.8681324},
    {
        "nombre": "BioFeria Municipalidad de Lurin",
        "lat": -12.2738587,
        "lon": -76.8697862,
    },
]

In [ ]:
bioferias_pd = pd.DataFrame.from_records(bioferias)
bioferias_geo = gpd.points_from_xy(bioferias_pd["lon"], bioferias_pd["lat"])
bioferias_gpd = gpd.GeoDataFrame(bioferias_pd, crs="EPSG:4326", geometry=bioferias_geo)
bioferias_gpd

## Ferias itinerantes de la MML

Organizadas por: Subgerencia de Desarrollo Productivo / Gerencia de Desarrollo Economico

In [ ]:
ferias_sdp = pd.read_excel(
    "/home/claudio/Downloads/FERIAS SDP - ESPACIOS - 2022.xlsx",
    header=5,
    index_col=0,
    usecols="B:I",
)

In [ ]:
ferias_sdp.head()

### Fix dates

In [ ]:
no_datetime_filter = ferias_sdp["Fecha"].apply(type) == str
ferias_sdp_no_dt = ferias_sdp["Fecha"][no_datetime_filter]
ferias_sdp_dt = ferias_sdp["Fecha"][~no_datetime_filter]

In [ ]:
ferias_sdp_no_dt

In [ ]:
# Handle special cases
special_case_1 = ferias_sdp_no_dt.loc[13]  # 10,11,12/06/2022
special_case_2 = ferias_sdp_no_dt.loc[24]  # 13 al 16/10/2022

special_case_start = pd.Series([("10/06/2022"), ("13/10/2022")], index=[13, 24])
special_case_end = pd.Series([("12/06/2022"), ("16/10/2022")], index=[13, 24])

ferias_sdp_no_dt = ferias_sdp_no_dt.drop([13, 24])

In [ ]:
# Separate start and end dates in to cols
start_dt = ferias_sdp_no_dt.str.split(" al ", expand=True)[0]
end_dt = ferias_sdp_no_dt.str.split(" al ", expand=True)[1]

In [ ]:
# Fix date format
start_dt.loc[9] = start_dt.loc[9].replace("22", "2022")
end_dt.loc[9] = end_dt.loc[9].replace("22", "2022")
end_dt.loc[23] = end_dt.loc[23].replace("1/", "01/")

In [ ]:
# Convert all datetimes
start_dt = pd.to_datetime(start_dt, format="%d/%m/%Y")
end_dt = pd.to_datetime(end_dt, format="%d/%m/%Y")
special_case_start = pd.to_datetime(special_case_start, format="%d/%m/%Y")
special_case_end = pd.to_datetime(special_case_end, format="%d/%m/%Y")
ferias_sdp_dt = pd.to_datetime(ferias_sdp_dt)

In [ ]:
# Concatenate all sets
start_dates_column = pd.concat(
    (ferias_sdp_dt, start_dt, special_case_start)
).sort_index()
end_dates_column = pd.concat((ferias_sdp_dt, end_dt, special_case_end)).sort_index()
dates_columns = pd.concat((start_dates_column, end_dates_column), axis=1)
dates_columns.columns = ["inicio", "fin"]

In [ ]:
# Merge with original df
ferias_sdp_dates_clean = ferias_sdp.merge(
    dates_columns, left_index=True, right_index=True
)

### Process coordinates

In [ ]:
ferias_sdp_gdf = gpd.GeoDataFrame(
    ferias_sdp_dates_clean,
    crs="EPSG:4326",
    geometry=gpd.points_from_xy(
        ferias_sdp_dates_clean["longitude"], ferias_sdp_dates_clean["latitude"]
    ),
)

In [ ]:
ferias_sdp_gdf.geometry.explore()

## Biotiendas:

### Flora y Fauna
- Flora y Fauna Miraflores, Avenida Mariscal La Mar 1110, -12.1108903, -77.0478323
- Flora y Fauna San Borja, Av. Aviación 3110, -12.1013079!4d-77.001902
- Flora y Fauna Surco, Av. Primavera 654, -12.1111641!4d-76.9865775
- Flora y Fauna La Molina, Av. Raúl Ferrero 1415, Lima 15024, Perú, -12.0915649!4d-76.9522467
- Flora y Fauna Miraflores 2*, Av. Alfredo Benavides 2150, Miraflores 15048, Perú, -12.1276912!4d-77.0101165
- Flora y Fauna San Miguel, Av. Universitaria 1045, -12.0753657, -77.0805019

### Vacas Felices
- Vacas Felices, Calle Alarifes Urb. Matellini MZ E-4 LT 63, Chorrillos, -12.1695999!4d-77.0076161
- Vacas Felices, JR. Los Lureles 106, Valle Hermoso, Surco, -12.126431!4d-76.9758164
- Vacas Felices, Juan Norberto Elespuru 305, San Isidro, -12.0965521!4d-77.0527479
- Vacas Felices, Barranco, Jr. Colina 108 (Alt. cuadra 5 de Av. Grau, espalda de la comisaria de Barranco, -12.1458844!4d-77.0222768
- Vacas Felices, San Isidro, Calle Alberto Arca Parró 194 -12.106375!4d-77.0364131

> \*No figura en la pagina web pero si en google maps.

In [ ]:
biotiendas = [
    [
        "Flora y Fauna",
        "Avenida Mariscal La Mar 1110, Miraflores",
        -12.1108903,
        -77.0478323,
    ],
    ["Flora y Fauna", "Avenida Aviación 3110, San Borja", -12.1013079, -77.001902],
    ["Flora y Fauna", "Avenida Primavera 654, Surco", -12.1111641, -76.9865775],
    ["Flora y Fauna", "Avenida Raúl Ferrero 1415, La Molina", -12.0915649, -76.9522467],
    [
        "Flora y Fauna",
        "Avenida Alfredo Benavides 2150, Miraflores",
        -12.1276912,
        -77.0101165,
    ],
    [
        "Flora y Fauna",
        "Avenida Universitaria 1045, San Miguel",
        -12.0753657,
        -77.0805019,
    ],
    [
        "Vacas Felices",
        "Calle Alarifes Urb. Matellini MZ E-4 LT 63, Chorrillos",
        -12.1695999,
        -77.0076161,
    ],
    [
        "Vacas Felices",
        "Jiron Los Lureles 106, Valle Hermoso, Surco",
        -12.126431,
        -76.9758164,
    ],
    [
        "Vacas Felices",
        "Juan Norberto Elespuru 305, San Isidro",
        -12.0965521,
        -77.0527479,
    ],
    ["Vacas Felices", "Jiron Colina 108, Barranco", -12.1458844, -77.0222768],
    [
        "Vacas Felices",
        "Calle Alberto Arca Parró 194, San Isidro",
        -12.106375,
        -77.0364131,
    ],
]

In [ ]:
biotiendas_pd = pd.DataFrame(biotiendas, columns=["nombre", "direccion", "lat", "lon"])
biotiendas_geo = gpd.points_from_xy(biotiendas_pd["lon"], biotiendas_pd["lat"])
biotiendas_gpd = gpd.GeoDataFrame(
    biotiendas_pd, crs="EPSG:4326", geometry=biotiendas_geo
)
biotiendas_gpd

## Supermercados con organicos

Aun falta determinar cuales son. Por el momento descargaremos todos los Wong y Tottus que se encuentren en OpenStreetMap (OSM)

In [ ]:
lima_met = up.download.nominatim_osm("Lima Metropolitana, Peru")

In [ ]:
lima_met = lima_met.set_crs("EPSG:4326")

In [ ]:
lima_met.total_bounds

In [ ]:
overpass_query = """
[timeout:120][out:json][bbox];
( 
  node[shop=supermarket][name~"Wong",i];
  way[shop=supermarket][name~"Wong",i];
  rel[shop=supermarket][name~"Wong",i];
);
out body geom;
"""

res = up.download.overpass_pois(
    bounds=lima_met.total_bounds, custom_query=overpass_query
)

In [ ]:
res.raise_for_status()

In [ ]:
data = res.json()

In [ ]:
osm_elements = pd.DataFrame.from_dict(data["elements"]).drop("tags", axis=1)

In [ ]:
node = osm_elements[osm_elements["type"] == "node"]
way = osm_elements[osm_elements["type"] == "way"]
rel = osm_elements[osm_elements["type"] == "relation"]

In [ ]:
len(rel)

In [ ]:
way = way.append(rel["members"].iloc[0], ignore_index=True)

In [ ]:
from shapely.geometry import Polygon

way["geometry"] = way["geometry"].apply(
    lambda x: Polygon(up.utils.shell_from_geometry(x))
)
way = way.drop_duplicates(subset=["geometry"])

In [ ]:
way_geometry = gpd.GeoSeries(way.geometry, crs="EPSG:4326")

In [ ]:
node_geometry = gpd.GeoSeries(
    gpd.points_from_xy(node["lon"], node["lat"], crs="EPSG:4326")
)

In [ ]:
ax = way_geometry.centroid.plot(color="red", alpha=0.4)
node_geometry.plot(alpha=0.4, ax=ax)

In [ ]:
supermarkets_gpd = gpd.GeoSeries(
    pd.concat((way_geometry.centroid, node_geometry), ignore_index=True)
)

In [ ]:
supermarkets_gpd

In [ ]:
supermarkets_gpd = supermarkets_gpd.drop(10)

In [ ]:
supermarkets_gpd.plot(alpha=0.5)

### Distritos

In [ ]:
districts = gpd.read_file(
    "https://storage.googleapis.com/up_public_geodata/admin_bounds/peru/districts.zip",
    mask=lima_met,
)

In [ ]:
all_points = pd.concat(
    [
        bioferias_gpd[["geometry"]],
        biotiendas_gpd[["geometry"]],
        ferias_sdp_gdf[["geometry"]],
    ]
)

In [ ]:
districts_with_points = districts.sjoin(all_points).drop_duplicates(subset="IDDIST")

In [ ]:
carabayllo = districts[districts["DISTRITO"] == "CARABAYLLO"]

In [ ]:
districts_with_points = districts_with_points.append(carabayllo)

### Vecindarios

In [ ]:
zona_carabayllo = gpd.read_file("inputs/zona_carabayllo.geojson", crs=4326)
zona_pachacamac = gpd.read_file("inputs/zona_pachacamac.geojson", crs=4326)

In [ ]:
from shapely.validation import make_valid

In [ ]:
zona_carabayllo.geometry = zona_carabayllo.geometry.apply(make_valid)
zona_pachacamac.geometry = zona_pachacamac.geometry.apply(make_valid)

In [ ]:
zona_carabayllo = gpd.GeoSeries(
    zona_carabayllo.geometry.unary_union.buffer(0.0001).exterior, crs=4326
)
zona_pachacamac = gpd.GeoSeries(
    zona_pachacamac.geometry.unary_union.buffer(0.0001).exterior, crs=4326
)

## Lima hexs

In [ ]:
lima_hexs = gpd.read_file("outputs/lima_hexs9_distance_to_bio.geojson")

In [ ]:
lima_hexs.shape

In [ ]:
lima_hexs.columns

## Generate maps

In [ ]:
UTM_CRS = "EPSG:32718"
lima_met_proj = lima_met.to_crs(UTM_CRS)
bioferias_gpd_proj = bioferias_gpd.to_crs(UTM_CRS)
biotiendas_gpd_proj = biotiendas_gpd.to_crs(UTM_CRS)
supermarkets_gpd_proj = supermarkets_gpd.to_crs(UTM_CRS)
ferias_sdp_gdf_proj = ferias_sdp_gdf.to_crs(UTM_CRS)
districts_with_points_proj = districts_with_points.to_crs(UTM_CRS)
zona_carabayllo_proj = zona_carabayllo.to_crs(UTM_CRS)
zona_pachacamac_proj = zona_pachacamac.to_crs(UTM_CRS)
lima_hexs_proj = lima_hexs.to_crs(UTM_CRS)
supermarkets_gpd_proj = supermarkets_gpd.to_crs(UTM_CRS)

In [ ]:
# Smooth borders
lima_met_smooth_proj = lima_met_proj.buffer(0.0001)

In [ ]:
districts_w_points_centroid = districts_with_points_proj.centroid

### Mapa de oferta alimentaria

Capas:  
- Puntos
- Limites distritales
- Vecindarios

In [ ]:
ax = lima_met_smooth_proj.plot(
    facecolor="none", edgecolor="lightgrey", linewidth=2, figsize=(10, 10)
)
districts_with_points_proj.plot(
    facecolor="none", edgecolor="grey", linewidth=1.5, ax=ax
)

texts = []
for x, y, label in zip(
    districts_w_points_centroid.geometry.x,
    districts_w_points_centroid.geometry.y,
    districts_with_points["DISTRITO"].str.title(),
):
    texts.append(plt.text(x, y, label, fontsize=5))

zona_carabayllo_proj.plot(
    facecolor="none", edgecolor="red", linewidth=1, zorder=1, ax=ax
)
zona_pachacamac_proj.plot(
    facecolor="none", edgecolor="red", linewidth=1, zorder=1, ax=ax
)

supermarkets_gpd_proj.plot(ax=ax, color="g", markersize=25)
bioferias_gpd_proj.plot(ax=ax, color="b", markersize=25)
biotiendas_gpd_proj.plot(ax=ax, color="orange", markersize=25)
ferias_sdp_gdf_proj.plot(ax=ax, color="c", markersize=25)

minx, miny, maxx, maxy = lima_met_smooth_proj.total_bounds
margins = 100
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)

ax.add_artist(ScaleBar(1, location="lower right"))

g_patch = Line2D([0], [0], marker="o", color="g", linewidth=0, label="Wong")
b_patch = Line2D([0], [0], marker="o", color="b", linewidth=0, label="BioFerias")
orange_patch = Line2D(
    [0], [0], marker="o", color="orange", linewidth=0, label="BioTiendas"
)
c_patch = Line2D(
    [0], [0], marker="o", color="c", linewidth=0, label="Ferias MML\nItinerantes"
)

r_line = Line2D([0], [0], color="r", linewidth=1, label="Vecindarios Alim.")
g_line = Line2D([0], [0], color="grey", linewidth=2, label="Distritos")
lg_line = Line2D([0], [0], color="lightgrey", linewidth=4, label="Provincia")

plt.legend(handles=[b_patch, orange_patch, c_patch, g_patch, r_line, g_line, lg_line])

ax.set_axis_off()
cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Positron, crs=UTM_CRS)

plt.savefig(
    "outputs/producto_2/maps/puntos_distritos_vecindarios.png",
    dpi=300,
    bbox_inches="tight",
)

### Mapa de oferta alimentaria

Capas:  
- Puntos
- Limites distritales
- Vecindarios
- Densidad

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

divider = make_axes_locatable(ax)

cax = divider.append_axes("right", size="5%", pad=0.1)

leg_kwds = {"label": "Población por 0.10$km^2$"}

ax = lima_hexs_proj.plot(
    "population_2020", cmap="Reds", legend=True, legend_kwds=leg_kwds, cax=cax, ax=ax
)


lima_met_smooth_proj.plot(facecolor="none", edgecolor="lightgrey", linewidth=2, ax=ax)
districts_with_points_proj.plot(
    facecolor="none", edgecolor="grey", linewidth=1.5, ax=ax
)

# texts = []
# for x, y, label in zip(districts_w_points_centroid.geometry.x, districts_w_points_centroid.geometry.y, districts_with_points["DISTRITO"].str.title()):
#    texts.append(plt.text(x, y, label, fontsize=5))

zona_carabayllo_proj.plot(
    facecolor="none", edgecolor="red", linewidth=1, zorder=1, ax=ax
)
zona_pachacamac_proj.plot(
    facecolor="none", edgecolor="red", linewidth=1, zorder=1, ax=ax
)
supermarkets_gpd_proj.plot(ax=ax, color="g", markersize=25)
bioferias_gpd_proj.plot(ax=ax, color="b", markersize=25, alpha=0.5)
biotiendas_gpd_proj.plot(ax=ax, color="orange", markersize=25, alpha=0.5)
ferias_sdp_gdf_proj.plot(ax=ax, color="c", markersize=25, alpha=0.5)

minx, miny, maxx, maxy = lima_met_smooth_proj.total_bounds
margins = 100
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)

ax.add_artist(ScaleBar(1, location="lower right"))
g_patch = Line2D([0], [0], marker="o", color="g", linewidth=0, label="Wong")
b_patch = Line2D([0], [0], marker="o", color="b", linewidth=0, label="BioFerias")
orange_patch = Line2D(
    [0], [0], marker="o", color="orange", linewidth=0, label="BioTiendas"
)
c_patch = Line2D(
    [0], [0], marker="o", color="c", linewidth=0, label="Ferias MML\nItinerantes"
)

r_line = Line2D([0], [0], color="r", linewidth=1, label="Vecindarios Alim.")
g_line = Line2D([0], [0], color="grey", linewidth=2, label="Distritos")
lg_line = Line2D([0], [0], color="lightgrey", linewidth=4, label="Provincia")

plt.legend(handles=[b_patch, orange_patch, c_patch, g_patch, r_line, g_line, lg_line])

ax.set_axis_off()
cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Positron, crs=UTM_CRS)

plt.savefig(
    "outputs/producto_2/maps/puntos_distritos_vecindarios_poblacion.png",
    dpi=300,
    bbox_inches="tight",
)

### Mapa de oferta alimentaria

Capas:  
- Puntos
- Limites distritales
- Vecindarios
- Pobreza

In [ ]:
lima_hexs_proj["NSE"] = lima_hexs_proj[
    ["Alto", "Bajo", "Medio", "Medio Alto", "Medio Bajo"]
].idxmax(axis=1)

In [ ]:
cat_order = ["Alto", "Medio Alto", "Medio", "Medio Bajo", "Bajo"]
ax = lima_hexs_proj.plot(
    "NSE", cmap="Reds", legend=True, categories=cat_order, alpha=0.45, figsize=(10, 10)
)

lima_met_smooth_proj.plot(facecolor="none", edgecolor="lightgrey", linewidth=2, ax=ax)
districts_with_points_proj.plot(
    facecolor="none", edgecolor="grey", linewidth=1.5, ax=ax
)

# texts = []
# for x, y, label in zip(districts_w_points_centroid.geometry.x, districts_w_points_centroid.geometry.y, districts_with_points["DISTRITO"].str.title()):
#    texts.append(plt.text(x, y, label, fontsize=5))

zona_carabayllo_proj.plot(
    facecolor="none", edgecolor="red", linewidth=1, zorder=1, ax=ax
)
zona_pachacamac_proj.plot(
    facecolor="none", edgecolor="red", linewidth=1, zorder=1, ax=ax
)
supermarkets_gpd_proj.plot(ax=ax, color="g", markersize=25)
bioferias_gpd_proj.plot(ax=ax, color="b", markersize=25, alpha=0.5)
biotiendas_gpd_proj.plot(ax=ax, color="orange", markersize=25, alpha=0.5)
ferias_sdp_gdf_proj.plot(ax=ax, color="c", markersize=25, alpha=0.5)

minx, miny, maxx, maxy = lima_met_smooth_proj.total_bounds
margins = 100
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)

ax.add_artist(ScaleBar(1, location="lower right"))
g_patch = Line2D([0], [0], marker="o", color="g", linewidth=0, label="Wong")
b_patch = Line2D([0], [0], marker="o", color="b", linewidth=0, label="BioFerias")
orange_patch = Line2D(
    [0], [0], marker="o", color="orange", linewidth=0, label="BioTiendas"
)
c_patch = Line2D(
    [0], [0], marker="o", color="c", linewidth=0, label="Ferias MML\nItinerantes"
)

r_line = Line2D([0], [0], color="r", linewidth=1, label="Vecindarios Alim.")
g_line = Line2D([0], [0], color="grey", linewidth=2, label="Distritos")
lg_line = Line2D([0], [0], color="lightgrey", linewidth=4, label="Provincia")

new_handlers = [b_patch, orange_patch, c_patch, g_patch, r_line, g_line, lg_line]
new_labels = [p.get_label() for p in new_handlers]

leg = ax.get_legend()
handlers = leg.legendHandles
labels = [text.get_text() for text in leg.texts]

ax.legend(
    handlers + new_handlers, labels + new_labels, title="INEI - Nivel\nSocioeconómico"
)

ax.set_axis_off()
cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Positron, crs=UTM_CRS)

plt.savefig(
    "outputs/producto_2/maps/puntos_distritos_vecindarios_nse.png",
    dpi=300,
    bbox_inches="tight",
)

### Mapa de oferta alimentaria

Capas:  
- Puntos
- Limites distritales
- Vecindarios
- Accessibilidad

In [ ]:
lima_hexs_proj["duration_to_nearest_poi_label"].unique()

In [ ]:
cat_order = [
    "Más de 120",
    "De 90 a 120",
    "De 60 a 90",
    "De 45 a 60",
    "De 30 a 45",
    "De 15 a 30",
    "De 0 a 15",
]

dff = lima_hexs_proj.query("population_2020 > 0").query(
    "duration_to_nearest_poi_label != 'nan'"
)
ax = dff.plot(
    "duration_to_nearest_poi_label",
    cmap="magma",
    legend=True,
    categories=cat_order,
    alpha=0.6,
    figsize=(10, 10),
)

lima_met_smooth_proj.plot(facecolor="none", edgecolor="lightgrey", linewidth=2, ax=ax)
districts_with_points_proj.plot(
    facecolor="none", edgecolor="grey", linewidth=1.5, ax=ax
)

# texts = []
# for x, y, label in zip(districts_w_points_centroid.geometry.x, districts_w_points_centroid.geometry.y, districts_with_points["DISTRITO"].str.title()):
#    texts.append(plt.text(x, y, label, fontsize=5))

zona_carabayllo_proj.plot(
    facecolor="none", edgecolor="red", linewidth=1, zorder=1, ax=ax
)
zona_pachacamac_proj.plot(
    facecolor="none", edgecolor="red", linewidth=1, zorder=1, ax=ax
)
supermarkets_gpd_proj.plot(ax=ax, color="g", markersize=25)
bioferias_gpd_proj.plot(ax=ax, color="b", markersize=25, alpha=0.5)
biotiendas_gpd_proj.plot(ax=ax, color="orange", markersize=25, alpha=0.5)
ferias_sdp_gdf_proj.plot(ax=ax, color="c", markersize=25, alpha=0.5)

minx, miny, maxx, maxy = lima_met_smooth_proj.total_bounds
margins = 100
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)

ax.add_artist(ScaleBar(1, location="lower right"))
g_patch = Line2D([0], [0], marker="o", color="g", linewidth=0, label="Wong")
b_patch = Line2D([0], [0], marker="o", color="b", linewidth=0, label="BioFerias")
orange_patch = Line2D(
    [0], [0], marker="o", color="orange", linewidth=0, label="BioTiendas"
)
c_patch = Line2D(
    [0], [0], marker="o", color="c", linewidth=0, label="Ferias MML\nItinerantes"
)

r_line = Line2D([0], [0], color="r", linewidth=1, label="Vecindarios Alim.")
g_line = Line2D([0], [0], color="grey", linewidth=2, label="Distritos")
lg_line = Line2D([0], [0], color="lightgrey", linewidth=4, label="Provincia")

new_handlers = [b_patch, orange_patch, c_patch, g_patch, r_line, g_line, lg_line]
new_labels = [p.get_label() for p in new_handlers]

leg = ax.get_legend()
handlers = leg.legendHandles
labels = [text.get_text() for text in leg.texts]

ax.legend(
    handlers + new_handlers,
    labels + new_labels,
    title="Tiempo de viaje\ncaminando (min)",
)

ax.set_axis_off()
cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Positron, crs=UTM_CRS)

plt.savefig(
    "outputs/producto_2/maps/puntos_distritos_vecindarios_accesibilidad.png",
    dpi=300,
    bbox_inches="tight",
)

- cuantas son
- que frecuencia
- distancia 
- (accesibilidad) minutos caminando | autos (tiempo de viaje)

In [ ]:
bioferias_gpd.head()

In [ ]:
bioferias_gpd.columns

In [ ]:
biotiendas_gpd.head()

In [ ]:
biotiendas_gpd.columns

In [ ]:
ferias_sdp_gdf.head()

In [ ]:
ferias_sdp_gdf.columns

In [ ]:
[
    "Actividad",
    "inicio",
    "fin",
    "Lugar",
    "Referencia",
    "Google maps url",
    "latitude",
    "longitude",
    "geometry",
]

IDRC ($ 3200)
- Armar base unica de kobotoolbox 
- Limpiar base
- Priorizacion de variables
- Georreferenciar dataset vecindarios 
- Ejes de analisis espacial

CIP ($ 3000)
- Subir encuesta de Juan a kobotoolbox 
- Limpiar base 

Oferta alimentaria agroecologica
- Puntos
- Limites de distritos
- Pobreza
- Densidad de poblacion
- Vecindarios Alimentarios
 
- cuantas son
- que frecuencia
- distancia 
- (accesibilidad) minutos caminando | autos (tiempo de viaje)

In [ ]:
supermarkets_gpd

In [ ]:
supermarkets_gpd

In [ ]:
biopoints = pd.concat(
    [
        bioferias_gpd.geometry,
        biotiendas_gpd.geometry,
        ferias_sdp_gdf.geometry,
        supermarkets_gpd.geometry,
    ]
)

In [ ]:
biopoints.shape

In [ ]:
biopoints.to_file("outputs/data/bio_points_w_wong.geojson")